In [7]:
import ee
import datetime

In [8]:
# Initialize the Earth Engine library.
ee.Initialize()

In [12]:
# Define the area of interest (AOI)
aoi = ee.Geometry.Polygon([
    [
        [39.24781494969205, 21.88731024190632],  # Bottom Left
        [39.280945595444, 21.88731024190632],    # Bottom Right
        [39.280945595444, 21.916058795347105],   # Top Right
        [39.24781494969205, 21.916058795347105], # Top Left
        [39.24781494969205, 21.88731024190632]   # Closing the loop
    ]
])

# Function to calculate NDVI
def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))

# Function to mask clouds and shadows
def maskClouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

# Function to export images to Google Drive
def exportImage(image, index):
    date = image.date().format('YYYY-MM-dd').getInfo()  # Get the image date for naming
    task = ee.batch.Export.image.toDrive(
        image=image.select(['NDVI']),
        description=f'NDVI_{date}_{index}',
        scale=10,
        region=aoi.getInfo()['coordinates'],
        folder='YourFolderName',  # Specify your folder name here
        fileFormat='GeoTIFF'
    )
    task.start()

# Load Sentinel-2 ImageCollection, filter by date and AOI, apply NDVI and cloud masking
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2018-01-01', '2018-12-31') \
    .filterBounds(aoi) \
    .map(addNDVI) \
    .map(maskClouds)

# Convert the collection to a list for iteration
imageList = sentinelCollection.toList(sentinelCollection.size())

# Loop through the image list to export each image
for index in range(imageList.size().getInfo()):
    eeImage = ee.Image(imageList.get(index))
    exportImage(eeImage, index)

print("Export tasks started. Check the 'Tasks' tab in the GEE Code Editor or your Google Drive for progress.")


Export tasks started. Check the 'Tasks' tab in the GEE Code Editor or your Google Drive for progress.
